<figure>
  <IMG SRC="https://raw.githubusercontent.com/mbakker7/exploratory_computing_with_python/master/tudelft_logo.png" WIDTH=25% ALIGN="right">
</figure>

<p><div> 
<br><b>Probabilistic Design</b>
<br><b>CEGM2XXX</b>
<br> <i>This course is given at the faculty of Civil Engineering and Geosciences at the Technical University of Delft</i>
<br> <i>Add course description or maybe  the learning objectives,that will be touched upon this week.</i>
<br><b>Teaching team:</b>
<br><i>- Dr. Ir. R.C. Lanzafame</i>
</div>
<br>
<br><b>Notebook created by teaching assistant:</b>
<br><i>- Siemen Algra</i>
</div>


<h2 style="color:#00BFFF;">Assignment: -------------</h2>

### Introduction:  
*-----------------------*

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy.optimize import fsolve # for plotting the LSF

import openturns as ot
import openturns.viewer as viewer
ot.Log.Show(ot.Log.NONE)

import numpy as np
import scipy.linalg as scp
import matplotlib.pylab as plt
import time

import sympy as sp

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

In [ ]:
# ----------------------------------------- Creating matrix of the FEM model ----------------------------------------- #
# Mesh
DistLandings = 17000 # Distance between landings
Deviation = 2000 # The maximum deviation

r = sp.symbols('r')
equation = sp.Eq((DistLandings/2)**2 + (r - Deviation)**2, r**2)
solutions = sp.solve(equation, r)
TunRad = float(solutions[0]) #m

TunAng = (180-np.arccos((DistLandings/2)/TunRad)*180/np.pi, np.arccos((DistLandings/2)/TunRad)*180/np.pi) #deg

dth = 0.0509 #deg
TunCX = TunRad*np.cos( np.deg2rad( np.arange(TunAng[0], TunAng[1]-dth, -dth) ) ) + DistLandings/2
TunCY = TunRad*np.sin( np.deg2rad( np.arange(TunAng[0], TunAng[1]-dth, -dth) ) )
TunCY = TunCY - min(TunCY)

# Properties of the beam
a = 23.8                            #m
b = 9.2                             #m
A = 38                              #m2
E_c = 50e9                          #E-modulus of high strength concrete
J = (np.pi*a**3*b**3)/(a**2 + b**2) #Torsion constant of the beam
G = E_c / (2*(1+0.3))
rho_c = 2500                        #kg/m^3 of concrete

Beam_m = 150000                    # [kg/m]
Beam_EI = E_c*(np.pi*a**3*b)/4      # [N.m2]
Beam_EIz = E_c*(np.pi*a*b**3)/4
Beam_EA = E_c*A                    # [N]
Beam_GJ = G*J                      # [N.m2]
Beam_Im = rho_c*J                  #[m2]

# ------------------------------------------------ Creating the nodes ------------------------------------------------ #
NodeC = [ [x,y] for x,y in zip(TunCX, TunCY) ]
nNode = len(NodeC)


# Define elements (and their properties
#             NodeLeft    NodeRight          m         EA        EI
Ele = [ [n1, n2, Beam_m, Beam_EA, Beam_EI] 
           for n1,n2 in zip(range(0,nNode-1), range(1,nNode)) ]
nEle = len(Ele)



# ------------------------------------------------ Creating the matrix ----------------------------------------------- #
LDOF = 3
nDof = LDOF*nNode  #3 dof per node
K = np.zeros(nDof*nDof)
M = np.zeros(nDof*nDof)
Q = np.zeros(nDof)
from BeamMatrices_Siemen import Beam2DMatrices
    

exeTime = [0.0, 0.0]
exeTime[0] = time.time()

for iEle in range(0, nEle):
    n1, n2, m, EA, EI = Ele[iEle]
    
    x1, y1 = NodeC[n1]
    x2, y2 = NodeC[n2]
    
    length = np.sqrt((x2-x1)**2 + (y2-y1)**2)
    
    n1dof = LDOF*n1 + np.arange(0,LDOF)
    n2dof = LDOF*n2 + np.arange(0,LDOF)    
    
    # Calculate elemental matrices
    Me, Ke, alpha, T_transformation = Beam2DMatrices(m, EA, EI, (NodeC[n1], NodeC[n2]) )
    
    
    indexes = np.append(n1dof, n2dof)
    for i in range(0, 2*LDOF):
        for j in range(0, 2*LDOF):
            ij = indexes[i]*nDof + indexes[j]
            M[ij] = M[ij] + Me[i,j]
            K[ij] = K[ij] + Ke[i,j]
    

# Reshape the global matrix from a 1-dimensional array to a 2-dimensional array
M = M.reshape((nDof, nDof))
K = K.reshape((nDof, nDof))

# ------------------------------------------------ Applying the boundary conditions ----------------------------------- #
NodesClamp = (0, nNode-1)

# Prescribed dofs
DofsP = np.empty([0], dtype=int)
for n0 in NodesClamp:
    DofsP = np.append(DofsP, n0*LDOF + np.arange(0,LDOF))

# Free dofs
DofsF = np.arange(0, nDof)       # free DOFs
DofsF = np.delete(DofsF, DofsP)  # remove the fixed DOFs from the free DOFs array

# print(DofsP)
# print(DofsF)

M_FF = [ M[iRow,DofsF].tolist() for iRow in DofsF ]
K_FF = [ K[iRow,DofsF].tolist() for iRow in DofsF ]
np.shape(M_FF)

In [ ]:
# -------------------------------------------- Redoing some notation stuff ------------------------------------------- #
# free & fixed array indices
fx = DofsF[:, np.newaxis]
fy = DofsF[np.newaxis, :]
bx = DofsP[:, np.newaxis]
by = DofsP[np.newaxis, :]

# Mass
Mii = M[fx, fy]
Mib = M[fx, by]
Mbi = M[bx, fy]
Mbb = M[bx, by]

# Stiffness
Kii = K[fx, fy]
Kib = K[fx, by]
Kbi = K[bx, fy]
Kbb = K[bx, by]

# External force
Qi = Q[fx]
Qb = Q[bx]


# Rewriting some notation 
M_FP = Mib
K_FP = Kib

In [ ]:
# Forces and wave properties

In [ ]:
# Calculating drag force, assuming current and waves are in the same direction
def drag_force(omega, k, wave_height, u_current):
    rhoW = 1025 # kg/m^3
    CD = 0.13 # drag coefficient
    D = np.sqrt(38/np.pi)*2 # diameter of cylinder
    
    depth = 30 # depth of water
    Amplitude = wave_height/2 
    
    # This exponents makes everything go through the roof, idk why, actually well I do 
    u_waves = Amplitude*omega*np.exp(-k*depth)*1
    FD = 0.5 * rhoW * CD * D * (u_waves + u_current)**2
    # print(f'current velocity is {u_current} and wave velocity is {u_waves}')
    return FD

def drag_force_swell(omega, k, wave_height, u_current):
    rhoW = 1025 # kg/m^3
    CD = 0.13 # drag coefficient
    D = np.sqrt(38/np.pi)*2 # diameter of cylinder
    
    depth = 30 # depth of water
    Amplitude = wave_height/2 
    u_current = 0 
    # This exponents makes everything go through the roof, idk why, actually well I do 
    u_waves = Amplitude*omega*np.exp(-k*depth)*1
    FD = 0.5 * rhoW * CD * D * (u_waves)**2
    # print(f'current velocity is {u_current} and wave velocity is {u_waves}')
    return FD


# Calculating the intertial force/froude Krylov force
def inertia_force(omega, k, wave_height):
    rhoW = 1025                                 # kg/m^3
    CM = 1+((9.2/2) / (20.8/2))                                   # added mass coefficient
    D = np.sqrt(38/np.pi)*2                     # diameter of cylinder
    depth = 30                                  # depth of water
    omega = np.sqrt(9.81*k*np.tanh(k*depth))    # wave frequenc, from dispersion relation
    Amplitude = wave_height/2                   # wave amplitude
    
    # Since we are looking at static case and need to determine whether or not stresses are getting to high, it is sufficient to just use
    # the maximum of the cos function, which is 1
    cos_func = 1 #np.cos(omega*t-k*x)
    
    FK =  (np.pi*rhoW*CM*D**2)/4 * (Amplitude*omega**2*np.exp(-k*depth)*1)
    # print(f'wave acc. for intertia is {Amplitude*omega**2*np.exp(-k*depth)*1}')
    return FK

def forces_combined(omega, k, wave_height, u_current):
    FD = drag_force(omega, k, wave_height, u_current)
    FK = inertia_force(omega, k, wave_height)
    F_total = np.sqrt(FD**2 + FK**2)
    return F_total

def forces_combined_swell(omega, k, wave_height, u_current):
    FD = drag_force_swell(omega, k, wave_height, u_current)
    FK = inertia_force(omega, k, wave_height)
    F_total = np.sqrt(FD**2 + FK**2)
    return F_total

    

In [ ]:
def wave_properties(significant_wave_height):
    Tp = np.sqrt(32*np.pi*significant_wave_height/9.81)
    Omega = 2*np.pi/Tp
    L = 9.81*Tp**2/(2*np.pi)
    k = 2*np.pi/L
    return Tp, Omega, k

def wave_properties_swell(significant_wave_height):
    Tp = np.sqrt(32*np.pi*significant_wave_height/9.81)
    Tp = 12
    Omega = 2*np.pi/Tp
    L = 9.81*Tp**2/(2*np.pi)
    k = 2*np.pi/L
    return Tp, Omega, k

In [ ]:
def calculate_stress_FEM(Total_force):
    u_displ = scipy.linalg.solve(K_FF, Force_vector(Total_force))
    
    # Length of the element
    L_elem = 46.157	

    # Observation point
    x_obs = L_elem/2

    ux1 = np.array(u_displ[0::6])
    uy1 = np.array(u_displ[1::6])
    ry1 = np.array(u_displ[2::6])



    ux2 = np.array(u_displ[3::6])
    # ux2 = np.append(ux2, 0)

    uy2 = np.array(u_displ[4::6])
    # uy2 = np.append(uy2, 0)

    ry2 = np.array(u_displ[5::6])
    # ry2 = np.append(ry2, 0)

    Myk = Beam_EI/(L_elem**2) * ((-6+(12*x_obs/L_elem))*uy1 + (4*L_elem + 6*x_obs)*ry1 + -(-6+(12*x_obs/L_elem))*uy2 + (-3*L_elem + 6*x_obs)*ry2)

    Iy = (Beam_EI/E_c)

    sigma_M = ((Myk*(a/2))/Iy)*1e-6 #MPa
    return sigma_M
    

In [ ]:
def total_stress(wave_height_wind, wave_height_swell, u_current):
    # Calculating wave properties
    Tp_wind, Omega_wind, k_wind = wave_properties(wave_height_wind)
    Tp_swell, Omega_swell, k_swell = wave_properties_swell(wave_height_swell)
    
    # Calculate force
    Total_force_wind = forces_combined(Omega_wind, k_wind, wave_height_wind, u_current)
    Total_force_swell = forces_combined_swell(Omega_swell, k_swell, wave_height_swell, u_current)
    
    # Calculate stress using FEM
    sigma_wind = calculate_stress_FEM(Total_force_wind)
    sigma_swell = calculate_stress_FEM(Total_force_swell)
    
    # Total stress
    sigma = sigma_wind + sigma_swell 
    return sigma   